<a href="https://colab.research.google.com/github/LeoneFabio/Egocentric-Vision/blob/main/Video-LLaVa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pillow
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install av
!pip install numpy
!pip install bitsandbytes-cuda111



In [ ]:
from PIL import Image
import numpy as np
import av
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration, BitsAndBytesConfig
from huggingface_hub import hf_hub_download
import torch
import os
torch.cuda.is_available()  # Dovrebbe restituire True se la GPU è attiva


False

In [ ]:
def read_video_pyav(container, indices):

    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


In [ ]:
# Usa FP16 e GPU
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VideoLlavaForConditionalGeneration.from_pretrained(
    "LanguageBind/Video-LLaVA-7B-hf",
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)
# Carica il processore
processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")

# Definisci il prompt
prompt = "USER: <video>what do you see? ASSISTANT:"

#video_path = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")
video_path = '/content/sample-5s.mp4'
# Apri il video con PyAV
container = av.open(video_path)

# Campiona uniformemente 8 frame dal video
total_frames = container.streams.video[0].frames
# Campiona solo 2 frame dal video
indices = np.arange(0, total_frames, total_frames / 15).astype(int)  # Riduce i frame
clip = read_video_pyav(container, indices)


# Prepara input e genera risposta
inputs = processor(text=prompt, videos=clip, return_tensors="pt").to(device)
torch.cuda.empty_cache()
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
generate_ids = model.generate(**inputs, max_new_tokens=100)
print(processor.batch_decode(generate_ids, skip_special_tokens=True)[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-00001-of-00003.safetensors:  12%|#1        | 577M/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.81G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/66.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

KeyboardInterrupt: 